In [ ]:
#| default_exp setup

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# ez_kaggle.setup

> Foundational set up for kaggle api usage and config

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os, json
from pathlib import Path
import configparser

In [ ]:
#| hide
from fastcore.test import *

## General

In [ ]:
#|export
def in_kaggle():
    "Check if code is running in a kaggle kernel environment"
    iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
    if iskaggle: return True
    else: return False

In [ ]:
#|export
IN_KAGGLE = in_kaggle()

In [ ]:
#|export
def import_kaggle():
    "Import kaggle API, using Kaggle secrets `kaggle_username` and `kaggle_key` if needed"
    if IN_KAGGLE:
        from kaggle_secrets import UserSecretsClient
        sec = UserSecretsClient()
        os.environ['KAGGLE_USERNAME'] = sec.get_secret("kaggle_username")
        if not os.environ['KAGGLE_USERNAME']: raise Exception("Please insert your Kaggle username and key into Kaggle secrets")
        os.environ['KAGGLE_KEY'] = sec.get_secret("kaggle_key")
    from kaggle import api
    return api

In [ ]:
#|eval: false
api = import_kaggle()
res = api.competitions_list(search='titanic')
assert len(res) > 0
print(res)

[spaceship-titanic, titanic]


In [ ]:
#|export
def get_username(): 
    from kaggle import KaggleApi
    ka = KaggleApi()
    env = ka.read_config_file()
    conf = ka.read_config_environment()
    cred = conf if not env else env
    return cred['username']

## Competition

In [ ]:
#|export
def get_comp_data(competition):
    "Get a path to data for `competition`, downloading it if needed"
    if IN_KAGGLE:
        return Path('../input')/competition
    else:
        path = Path(competition)
        api = import_kaggle()
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition))
            zipfile.ZipFile(f'{competition}.zip').extractall(str(competition))
        return path

In [ ]:
#|eval: false
path = get_comp_data('titanic')
assert path == Path('titanic')
assert path.exists()
assert path.ls().sorted() == [Path('titanic/gender_submission.csv'),Path('titanic/test.csv'),Path('titanic/train.csv')]

In [ ]:
#| export
def competition_config(
    competition, # ie titanic
    data_path=None,
    dataset_username=None, #ie isaacflath
    model_dataset_name=None, #ie 'models-pawpularity'
    libraries_dataset_name=None, #ie 'libraries-pawpularity'
    required_libraries=None, #ie ['fastkaggle','fastai']
    pip_cmd = 'pip'
):
    
    config = configparser.ConfigParser()
    config['DEFAULT'] = {'competition': competition,
                         'pip_cmd': pip_cmd, 
                         'data_path': data_path if data_path else '.'}
              
    un = dataset_username if dataset_username else get_username()
    config['DEFAULT']['datasets_username'] = un
        
    model_ds = model_dataset_name if model_dataset_name else f"models-{competition}"
    config['DEFAULT']['model_dataset_name'] = model_ds
            
    libraries_ds = libraries_dataset_name if libraries_dataset_name else f"libraries-{competition}"
    config['DEFAULT']['libraries_dataset_name'] = libraries_ds
        
    libs = required_libraries if required_libraries else ['fastkaggle']
    config['DEFAULT']['required_libraries'] = ' '.join(libs)

    return config

In [ ]:
#|eval: false
cfg = competition_config('titanic')
test_eq([i for i in cfg['DEFAULT'].keys()],['competition', 'pip_cmd', 'data_path', 'datasets_username', 'model_dataset_name', 'libraries_dataset_name', 'required_libraries'])
print([i for i in cfg['DEFAULT'].items()])

[('competition', 'titanic'), ('pip_cmd', 'pip'), ('data_path', '.'), ('datasets_username', 'isaacflath'), ('model_dataset_name', 'models-titanic'), ('libraries_dataset_name', 'libraries-titanic'), ('required_libraries', 'fastkaggle')]


In [ ]:
#| export
def setup_comp(competition, # Name of compeition
               dataset_username=None, # username where datasets will be stored
               model_dataset_name=None, # name to store model weights
               libraries_dataset_name=None, # name to store libraries
               required_libraries=None, # needed libraries for competition
               pip_cmd='pip' # pip command to use for installation
              ):
    path = get_comp_data(competition)
    cfg = competition_config(competition,str(path.parent),dataset_username,model_dataset_name,libraries_dataset_name, required_libraries,pip_cmd)
    with open('fastkaggle.ini', 'w') as configfile: cfg.write(configfile)

In [ ]:
#|eval: false
setup_comp('titanic')
assert Path('fastkaggle.json').exists()

In [ ]:
cfg = configparser.ConfigParser()

In [ ]:
cfg.read('fastkaggle.ini')

['fastkaggle.ini']

In [ ]:
[i for i in cfg['DEFAULT'].items()]

[('competition', 'titanic'),
 ('pip_cmd', 'pip'),
 ('data_path', '.'),
 ('datasets_username', 'isaacflath'),
 ('model_dataset_name', 'models-titanic'),
 ('libraries_dataset_name', 'libraries-titanic'),
 ('required_libraries', 'fastkaggle')]

In [ ]:
cfg['DEFAULT'].update({'competition':'test'})

In [ ]:
#| export
def get_config_values(
    path='.', # path to kaggle.json file or None
    **cfg_overrides # config values (will override fastkaggle.json if exists
    ):
    cfg = configparser.ConfigParser()
    if path: cfg .read(Path(path)/'fastkaggle.ini')
    if cfg_overrides: cfg['DEFAULT'].update(cfg_overrides)
    return cfg

In [ ]:
#|eval: false
cfg = get_config_values() 
test_eq([o for o in cfg['DEFAULT'].keys()],['competition', 'pip_cmd', 'data_path', 'datasets_username', 'model_dataset_name', 'libraries_dataset_name', 'required_libraries'])
print([i for i in cfg['DEFAULT'].items()])

[('competition', 'titanic'), ('pip_cmd', 'pip'), ('data_path', '.'), ('datasets_username', 'isaacflath'), ('model_dataset_name', 'models-titanic'), ('libraries_dataset_name', 'libraries-titanic'), ('required_libraries', 'fastkaggle')]


In [ ]:
#|eval: false
cfg = get_config_values(competition='123',pip_cmd='4') 
assert cfg['DEFAULT'].getint('competition') == 123
assert cfg['DEFAULT'].get('pip_cmd') == '4'
print([i for i in cfg['DEFAULT'].items()])

[('competition', '123'), ('pip_cmd', '4'), ('data_path', '.'), ('datasets_username', 'isaacflath'), ('model_dataset_name', 'models-titanic'), ('libraries_dataset_name', 'libraries-titanic'), ('required_libraries', 'fastkaggle')]


## Export -

In [ ]:
#|hide
from nbdev.doclinks import nbdev_export
nbdev_export()